# Walker Texas Wrangle

## Exercises I - Repo Preparation

- Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
- Clone this repository within your local codeup-data-science directory.
- Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
- Ceate a README.md file that outlines the contents and purpose of your repository.
- Add, commit, and push these two files.
- Now you can add your env.py file to this repository to access the Codeup database server.
- For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.
- As always, add, commit, and push your work often.

## Exercise II - Acquire and Prep (Wrangle)

### Exercise Intro

- Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

- As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

In [ ]:
def get_zillow(user=user, password=password, host=host):
    """
    This function wrangles data from a cached CSV or SQL database, drops unneeded columns and null
    values, and returns a cleaned dataframe of Zillow property data.

    :param user: The username for accessing the MySQL database
    :param password: Please make sure to keep your password secure and not share it with anyone
    :param host: The host parameter is the address of the server where the MySQL database is hosted
    :return: a pandas DataFrame containing cleaned and wrangled data from the Zillow database for single
    family residential properties. The DataFrame includes columns for bedroom count, bathroom count,
    calculated finished square footage, tax value in dollars, year built, tax amount, and FIPS code. The
    function drops unneeded columns and null values before returning the DataFrame.
    """
    # name of cached csv
    filename = "zillow.csv"
    # if cached data exist
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    # wrangle from sql db if not cached
    else:
        # read sql query into df
        # 261 is single family residential id
        df = pd.read_sql(
            """select yearbuilt
                                    , bedroomcnt
                                    , bathroomcnt
                                    , calculatedfinishedsquarefeet
                                    , taxvaluedollarcnt
                                    , taxamount
                                    , fips 
                            from properties_2017
                            where propertylandusetypeid = 261""",
            f"mysql+pymysql://{user}:{password}@{host}/zillow",
        )
        # cache data locally
        df.to_csv(filename, index=False)
    # nulls account for less than 1% so dropping
    df = df.dropna()
    # rename columns
    df = df.rename(
        columns=(
            {
                "yearbuilt": "year",
                "bedroomcnt": "beds",
                "bathroomcnt": "baths",
                "calculatedfinishedsquarefeet": "area",
                "taxvaluedollarcnt": "tax_value",
                "taxamount": "prop_tax",
                "fips": "county",
            }
        )
    )
    # map county to fips
    df.county = df.county.map({6037: "LA", 6059: "Orange", 6111: "Ventura"})
    # make int
    ints = ["year", "beds", "area", "tax_value"]
    for i in ints:
        df[i] = df[i].astype(int)
    return df

### 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.



In [1]:
import pandas as pd
import env as e
import os


def get_zillow():
    # name of cached csv
    filename = "zillow.csv"
    # if cached data exist
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    # wrangle from sql db if not cached
    else:
        # read sql query into df
        # 261 is single family residential id
        df = pd.read_sql(
            """SELECT yearbuilt,
                                   bedroomcnt,
                                   bathroomcnt,
                                   calculatedfinishedsquarefeet,
                                   taxvaluedollarcnt,
                                   taxamount,
                                   fips
                            FROM properties_2017
                            WHERE propertylandusetypeid = 261""",
            f"mysql+pymysql://{e.user}:{e.password}@{e.host}/zillow",
        )
        # cache data locally
        df.to_csv(filename, index=False)
    return df

#### Exploration Summary Function

In [27]:
def check_columns(df_telco):
    """
    This function takes a pandas dataframe as input and returns
    a dataframe with information about each column in the dataframe. For
    each column, it returns the column name, the number of
    unique values in the column, the unique values themselves,
    the number of null values in the column, the proportion of null values,
    and the data type of the column. The resulting dataframe is sorted by the
    'Number of Unique Values' column in ascending order.

    Args:
    - df_telco: pandas dataframe

    Returns:
    - pandas dataframe
    """
    data = []
    # Loop through each column in the dataframe
    for column in df_telco.columns:
        # Append the column name, number of unique values, unique values, number of null values, proportion of null values, and data type to the data list
        data.append(
            [
                column,
                df_telco[column].nunique(),
                df_telco[column].unique(),
                df_telco[column].isna().sum(),
                df_telco[column].isna().mean(),
                df_telco[column].dtype,
            ]
        )
    # Create a pandas dataframe from the data list, with column names 'Column Name', 'Number of Unique Values', 'Unique Values', 'Number of Null Values', 'Proportion of Null Values', and 'dtype'
    # Sort the resulting dataframe by the 'Number of Unique Values' column in ascending order
    return pd.DataFrame(
        data,
        columns=[
            "Column Name",
            "Number of Unique Values",
            "Unique Values",
            "Number of Null Values",
            "Proportion of Null Values",
            "dtype",
        ],
    ).sort_values(by="Number of Unique Values")

### 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.



In [2]:
df = get_zillow()

In [33]:
info = check_columns(df)

info

,Column Name,Number of Unique Values,Unique Values,Number of Null Values,Proportion of Null Values,dtype
6,county,3,"[6037.0, 6059.0, 6111.0]",0,0.000000,float64
1,beds,19,"[0.0, 4.0, 3.0, 5.0, 2.0, 1.0, 6.0, 7.0, 8.0, ...",11,0.000005,float64
2,baths,38,"[0.0, 2.0, 4.0, 1.0, 2.5, 3.5, 3.0, 5.5, 4.5, ...",11,0.000005,float64
0,year,153,"[nan, 2005.0, 2011.0, 1926.0, 1972.0, 1973.0, ...",9337,0.004337,float64
3,sqfeet,10580,"[nan, 3633.0, 1620.0, 2077.0, 1200.0, 171.0, 2...",8484,0.003941,float64
4,tax_value,592269,"[27516.0, 10.0, 2108.0, 296425.0, 124.0, 84777...",493,0.000229,float64
5,prop_tax,918838,"[nan, 174.21, 6941.39, 10244.94, 7924.68, 8034...",4442,0.002063,float64


In [31]:
info["Proportion of Null Values"].sum()

0.010580329542567268

<pre>
When Googling fips, 6037 = Los Angeles County
                    6059 = Orange County
                    6111 = Midland County

- With this info, we'll rename 'fips' to 'county'

- We'll rename 'bedroomcnt' to 'beds'
               'bathroomcnt' to 'baths'
               'yearbuilt' to 'year'
               'calculatedfinishedsquarefeet' to 'sqfeet'
               'taxvaluedollarcnt' to 'tax_value'
               'taxamount' to 'prop_tax'

- With all of the proportion of null values adding up to 0.0105 (1%), we will drop all NA in the DF

In [12]:
df = df.rename(
    columns=(
        {
            "yearbuilt": "year",
            "bedroomcnt": "beds",
            "bathroomcnt": "baths",
            "calculatedfinishedsquarefeet": "sqfeet",
            "taxvaluedollarcnt": "tax_value",
            "taxamount": "prop_tax",
            "fips": "county",
        }
    )
)

In [12]:
df.dropna()

,year,beds,baths,sqfeet,tax_value,prop_tax,county
4,2005.0,4.0,2.0,3633.0,296425.0,6941.39,6037.0
6,2011.0,3.0,4.0,1620.0,847770.0,10244.94,6037.0
7,1926.0,3.0,2.0,2077.0,646760.0,7924.68,6037.0
11,1972.0,0.0,0.0,1200.0,5328.0,91.60,6037.0
14,1973.0,0.0,0.0,171.0,6920.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,2015.0,4.0,4.0,4375.0,422400.0,13877.56,6037.0
2152858,2015.0,4.0,3.0,2262.0,960756.0,13494.52,6059.0
2152859,2014.0,4.0,4.5,3127.0,536061.0,6244.16,6059.0
2152861,2015.0,3.0,2.5,1974.0,424353.0,5302.70,6059.0


### 3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow


In [14]:
import os
import pandas as pd
import env as e


def wrangle_zillow():
    """
    This function reads the Zillow data from a cached CSV file if it exists,
    or from a SQL database if it doesn't exist. It then renames the columns
    to more descriptive names.

    Args:
    - None

    Returns:
    - pandas dataframe
    """
    # Name of cached CSV file
    filename = "zillow.csv"
    # If cached data exists, read from CSV file
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    # Otherwise, read from SQL database
    else:
        df = pd.read_sql(
            """SELECT yearbuilt,
                                   bedroomcnt,
                                   bathroomcnt,
                                   calculatedfinishedsquarefeet,
                                   taxvaluedollarcnt,
                                   taxamount,
                                   fips
                            FROM properties_2017
                            WHERE propertylandusetypeid = 261""",  # 261 is single family residential id
            f"mysql+pymysql://{e.user}:{e.password}@{e.host}/zillow",
        )
        # Cache data locally
        df.to_csv(filename, index=False)
    # Rename columns
    df = df.rename(
        columns={
            "yearbuilt": "year",
            "bedroomcnt": "beds",
            "bathroomcnt": "baths",
            "calculatedfinishedsquarefeet": "sqfeet",
            "taxvaluedollarcnt": "tax_value",
            "taxamount": "prop_tax",
            "fips": "county",
        }
    )
    # Drop nulls, since it is less than 1%
    # Before Drop NA: 2,152,863
    # After Drop NA: 2,140,235
    # Total Dropped: 12,628 (0.006)
    df = df.dropna()
    return df

## Notes

### Use as reference for get_zillow_data

```python
def get_telco_data():
    """
    get telco data will query the telco database and return all the relevant churn data within

    arguments: none

    return: a pandas dataframe
    """
    filename = "telco.csv"
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        query = """
        SELECT *
        FROM customers
        JOIN contract_types
        ON customers.contract_type_id = contract_types.contract_type_id
        JOIN internet_service_types
        ON customers.internet_service_type_id = internet_service_types.internet_service_type_id
        JOIN payment_types
        ON customers.payment_type_id = payment_types.payment_type_id;"""
        connection = db_url("telco_churn")
        df = pd.read_sql(query, connection)
        df.to_csv(filename, index=False)
    return df
 
def check_columns(df):
    """
    This function takes in a pandas DataFrame and prints out the name of each column,
    the number of unique values in each column, and the unique values themselves.

    Parameters:
    -----------
    df : pandas DataFrame
        The DataFrame to be checked.
    """
    for column in df.columns:
        print(f"{column} ({df[column].nunique()})")
        print(f"Unique Values: {df[column].unique()}")
        print("")
```

### Convertting entire columns.

- **Useful for converting columns, and then investigating NA values**

- *May be useful to not run inplace=True or not assign it right away, to check out the funky value beforehand*

```python
pd.to_datetime()
#  This method converts a pandas object to a datetime type. The errors parameter can be set to 'raise' to raise a ValueError if the input data is not in a recognized format, or 'coerce' to return NaT (Not a Time) for non-convertible data. 

pd.to_timedelta() 
# This method converts a pandas object to a timedelta type. The errors parameter can be set to 'raise' to raise a ValueError if the input data is not in a recognized format, or 'coerce' to return NaT (Not a Time) for non-convertible data.

pd.to_bool()
# This method converts a pandas object to a boolean type. The errors parameter can be set to 'raise' to raise a ValueError if the input data is not a recognized boolean value, 'coerce' to return NaN for non-convertible data, or 'ignore' to leave non-convertible data as is.

pd.to_string()
# This method converts a pandas object to a string type. This method does not have an errors parameter.

pd.to_categorical()
# This method converts a pandas object to a categorical type. The errors parameter can be set to 'raise' to raise a ValueError if the input data is not a recognized category, or 'ignore' to leave non-convertible data as is.
```